In [7]:
import os
import pickle

In [8]:
# gmail api utils
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [9]:
# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode, urlsafe_b64encode

In [10]:
# for dealing with attachement MIME types
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from mimetypes import guess_type as guess_mime_type

In [11]:
# request all email permissions
SCOPES = ['https://mail.google.com/']
our_email = 'raitec@dee.ufc.br'

In [12]:
# function to load credentials.txt
def gmail_authenticate():
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

# get the Gmail API service
service = gmail_authenticate()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=146723282850-64198rf9a0g17tbsdcu54urb0u4hdppa.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58751%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=HJg1HDprNStMpq4YCMD0UckHYo4f76&access_type=offline


In [13]:
# Adds the attachment with the given filename to the given message
def add_attachment(message, filename):
    content_type, encoding = guess_mime_type(filename)
    if content_type is None or encoding is not None:
        content_type = 'application/octet-stream'
    main_type, sub_type = content_type.split('/', 1)
    if main_type == 'text':
        fp = open(filename, 'rb')
        msg = MIMEText(fp.read().decode(), _subtype=sub_type)
        fp.close()
    elif main_type == 'image':
        fp = open(filename, 'rb')
        msg = MIMEImage(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'audio':
        fp = open(filename, 'rb')
        msg = MIMEAudio(fp.read(), _subtype=sub_type)
        fp.close()
    else:
        fp = open(filename, 'rb')
        msg = MIMEBase(main_type, sub_type)
        msg.set_payload(fp.read())
        fp.close()
    filename = os.path.basename(filename)
    msg.add_header('Content-Disposition', 'attachment', filename=filename)
    message.attach(msg)

In [14]:
# function that takes some parameters and builds email message
def build_message(destination, obj, body, attachments=[]):
    if not attachments: # no attachments given
        message = MIMEText(body)
        message['to'] = destination
        message['from'] = our_email
        message['subject'] = obj
    else:
        message = MIMEMultipart()
        message['to'] = destination
        message['from'] = our_email
        message['subject'] = obj
        message.attach(MIMEText(body))
        for filename in attachments:
            add_attachment(message, filename)
    return {'raw': urlsafe_b64encode(message.as_bytes()).decode()}

In [15]:
# function that sends the message
def send_message(service, destination, obj, body, attachments=[]):
    return service.users().messages().send(
      userId="me",
      body=build_message(destination, obj, body, attachments)
    ).execute()

Now, we're done setting up Gmail API. Let's start the second part by reading a Google Sheet that will contain the schools' name and email

In [18]:
import pandas as pd

filepath = "https://docs.google.com/spreadsheets/d/1QKThZF7CxoHhdp4HNj1gjJOb0YfsUj2BEeY0-2PlJi0/export?format=csv&gid=0"

contact_list = pd.read_csv(filepath)
contact_list

,nome,e-mail
0,Leonardo,leonardodayvison235@gmail.com
1,Nataniel,natanielmarques@alu.ufc.br
2,Davi,david.jesseh@alu.ufc.br
3,Maria Paula,mariapaulamesquita@alu.ufc.br


In [23]:
def send_single_email(row):
    email_body = "Olá, {}! Muito obrigado por verificar o e-mail. Espero que isso esteja mesmo funcionando, pq acho que, se der errado, eu vou ter um treco.".format(row['nome'])
    
    send_message(service, row['e-mail'], "Teste de mensagem automática do DTEC", 
                 email_body, ['assinatura_dtec.png'])

In [30]:
import time 

def send_all_emails(df):
    for _, row in df.iterrows():
        try:
            send_single_email(row)
            time.sleep(2)
        except Exception as e:
            print(f"Error trying to send message to {row['e-mail']}: {e}")

IT'S THE TRUTH TIME GUYS

In [32]:
# uncomment the code below and run this cell to send email to all people up there on the contact sheet

#send_all_emails(contact_list)